In [1]:
import pandas as pd
from pathlib import Path

In [2]:
"""Importing the required libraries"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.layers import *
from keras import backend, Model


2025-08-18 13:55:14.702561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755525314.716399       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755525314.720415       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 13:55:14.736799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [3]:
"""Reading the training dataset and performing the EDA ( Exploratory Data Analysis ) in the upcoming cells"""

path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
dataset = pd.read_csv('../input/train.csv')


In [4]:
"""Performing EDA ( Exploratory Data Analysis ) """


#Having a look at the data
print("Number of rows in data = ", dataset.shape[0])
print("Number of columns in data = ", dataset.shape[1])
print("\n")
print("---Sample data---")
dataset.head()


Number of rows in data =  159571
Number of columns in data =  8


---Sample data---


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
"""Performing EDA ( Exploratory Data Analysis ) """

#Visualising the Training data : Number of comments vs. Label
labels = dataset.columns.values[2:]
labels_count = dataset.iloc[:, 2:].sum().values
sns.set(font_scale = 2)
plt.figure(figsize = (15, 8))
fig = sns.barplot(labels, labels_count)
plt.title("Comments vs. Label", fontsize = 24)
plt.ylabel('Number of comments', fontsize = 20)
plt.xlabel('Label ', fontsize = 20)
rects = fig.patches
for rect, label in zip(rects, labels_count):
    height = rect.get_height()
    fig.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha = 'center', va = 'bottom', fontsize = 18)
plt.show()


TypeError: barplot() takes from 0 to 1 positional arguments but 2 were given

<Figure size 1500x800 with 0 Axes>

In [6]:
"""Function for Text Preprocessing"""

#Removing non-english symbols, HTML tags, converting to lower-case, lemmatizing, and finally removing the stop-words 
stop_words = set(stopwords.words("english")) 
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
lemmatizer = WordNetLemmatizer()

def clean_text(X):
    processed = []
    for text in X:
        text = text[0]
        text = re.sub(r'[^\w\s]', '',text, re.UNICODE)
        text = re.sub('\n', ' ',text, re.UNICODE)
        text = re.sub('', '', text)
        text = text.lower()
        text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
        text = [lemmatizer.lemmatize(token, "v") for token in text]
        text = [word for word in text if not word in stop_words]
        processed.append(text)
    return processed


In [7]:
"""Getting the X and Y and preprocessing them"""

X = dataset.iloc[:, 1:2].values
y_train = dataset.iloc[:, 2:8].values
X_train = clean_text(X)


In [8]:
"""Tokenization and Padding"""
vocab_size = 10000
maxlen = 250
embed_dim = 20
batch_size = 64
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenized_word_list = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')


NameError: name 'Tokenizer' is not defined

In [9]:
"""EarlyStopping and ModelCheckpoint"""

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 2)
mc = ModelCheckpoint('model_best.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)


NameError: name 'EarlyStopping' is not defined

In [10]:
"""Creating TextCNN model for comment classification"""

#Defining the Layers of the model
input_X = Input(shape=(maxlen, ))
embed = Embedding(vocab_size, embed_dim)(input_X)
conv_1 = Conv1D(filters = 64, kernel_size = 3, activation = 'relu', padding = 'valid')(embed)
out_1 = GlobalMaxPooling1D()(conv_1)
conv_2 = Conv1D(filters = 64, kernel_size = 5, activation = 'relu', padding = 'valid')(embed)
out_2 = GlobalMaxPooling1D()(conv_2)
conc = concatenate([out_1, out_2])
dense1 = Dense(32, activation = 'relu')(conc)
out = Dense(6, activation = 'sigmoid', name = 'output_layer')(dense1)

#Defining the model now
model = Model(input_X, out)
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()


NameError: name 'Input' is not defined

In [11]:
"""Fitting the model"""
X_train, X_val, y_train, y_val = train_test_split(X_train_padded, y_train, test_size = 0.2, random_state = 42, shuffle = True)
model.fit(X_train, y_train, epochs = 10, batch_size = batch_size, verbose = 1, validation_data = [X_val, y_val], callbacks = [es, mc])


NameError: name 'X_train_padded' is not defined

In [12]:
"""Importing the Test Data and making it ready to be passed to the Model"""

dataset2 = pd.read_csv('../input/test.csv')
X_test = dataset2.iloc[:, 1:2].values
X_test = clean_text(X_test)
tokenized_word_list = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')


NameError: name 'tokenizer' is not defined

In [13]:
"""Testing and creating the test results"""

labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
model = load_model('model_best.h5')
y_test = model.predict(X_test_padded, batch_size = 512, verbose = 1)
sample_submission = pd.read_csv(f'{path}sample_submission.csv')
sample_submission[labels] = y_test
sample_submission.to_csv('submission.csv', index = False)


NameError: name 'load_model' is not defined